# **Задача**

В конце 2017 года платформа Civil Comments закрылась и опубликовала около 2 миллионов комментариев из социальных сетей, чтобы специалисты по данным со всего мира могли работать вместе над исследованием способов смягчения предвзятости в текстовых данных.

Таблица с комментариями также прилагается: data.csv. Мы будем работать с тысячами комментариев, где каждый комментарий помечен как «токсичный» или «нетоксичный».

В таблице data.csv в колонке comment_text написаны сами комментарии, с которыми нам предстоит работать. В колонке target стоят вероятности того, что комментарий токсичен. Давайте сделаем предпосылку, что комментарий считается токсичным, если вероятность выше 0.7.

Выполняем код ниже, чтобы отделить колонки и преобразовать вероятности в бинарные величины, где 1 — токсичный комментарий, а 0 — нетоксичный комментарий.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Get the same results each time
np.random.seed(0)


# Load the training data
data = pd.read_csv("D:/mifi/myvenv/ML_ethic/data.csv")
comments = data["comment_text"]
target = (data["target"]>0.7).astype(int) # Преобразуем вероятности в бинарные метки: 1, если target > 0.7, иначе 0

### **Исследование датасетов**

In [104]:
data.head(5)

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,...,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
2,239612,0.830769,This bitch is nuts. Who would read a book by a...,0.107692,0.661538,0.338462,0.830769,0.000000,0.0,0.0,...,26674,rejected,0,0,0,0,0,0.061538,4,65
5,240400,0.833333,"Nincompoop, that's a nice one! I'm partial to ...",0.000000,0.000000,0.000000,0.833333,0.000000,0.0,0.0,...,32846,approved,0,0,0,0,0,0.000000,4,6
11,240879,0.777778,"""Real freedom""? Freedom to kill wantonly when ...",0.049383,0.283951,0.024691,0.753086,0.111111,0.0,0.0,...,33626,approved,0,0,0,3,0,0.061728,4,81
23,242779,0.728571,"Wow, you wouldn't know a FACT, about the Bundy...",0.071429,0.657143,0.057143,0.571429,0.042857,0.0,0.0,...,37903,rejected,0,0,0,0,0,0.585714,4,70
25,243373,0.850000,Notwithstanding the fact the protester is an i...,0.016667,0.266667,0.016667,0.850000,0.000000,0.0,0.0,...,38031,approved,0,0,0,3,0,0.000000,4,60


In [105]:
target.head()

2     1
5     1
11    1
23    1
25    1
Name: target, dtype: int64

# **Задание 1**
Теперь разделим наши данные на train и test. Пусть в тест у нас пойдет 30% данных. Для этого можете использовать библиотеку train_test_split из sklearn.

In [106]:
comm_train, comm_test, target_train, target_test = train_test_split(
    comments, target, test_size=0.3, random_state=0
)

# Инорфмация о тренировочном наборе
print("Размер тренировочного датасета:", len(comm_train))
comm_train.info()

Размер тренировочного датасета: 7275
<class 'pandas.core.series.Series'>
Index: 7275 entries, 69855 to 22996
Series name: comment_text
Non-Null Count  Dtype 
--------------  ----- 
7275 non-null   object
dtypes: object(1)
memory usage: 113.7+ KB


# **Задание 2**

Как вы уже, наверное, догадались, в наших данных есть определенная проблема — это разные типы данных. Предсказывать мы хотим бинарную величину: 1 и 0. А на вход мы подаем текст, а не числа. Поэтому нам нужно как-то сконвертировать текст в число.

In [107]:
vectorizer = CountVectorizer()

# Обучаем векторизатор и преобразуем
comments_train_vec = vectorizer.fit_transform(comm_train)

# Применяем обученный векторизатор 
comments_test_vec = vectorizer.transform(comm_test)

print(f"Размерность обучающей выборки после векторизации: {comments_train_vec.shape}")
print(f"Размерность тестовой выборки после векторизации: {comments_test_vec.shape}")


Размерность обучающей выборки после векторизации: (7275, 20120)
Размерность тестовой выборки после векторизации: (3118, 20120)


# **Задание 3**
Теперь в качестве модели, которая будет классифицировать нам комментарии на токсичные и нетоксичные, возьмем логистическую регрессию.

In [108]:
model = LogisticRegression(max_iter=2000)
model.fit(comments_train_vec, target_train)

# предсказания на тестовой выборке
predictions = model.predict(comments_test_vec)

# Сaccuracy
accuracy = accuracy_score(target_test, predictions)

print(f"Accuracy test: {accuracy:.4f}")

Accuracy test: 0.8855


# **Задание 4**

Чтобы мы смогли протестировать разные комментарии, которые приходят в голову, пропишите ниже функцию, для которой на вход мы бы подавали наш комментарий, а на выход получали предсказание, насколько от 0 до 1 комментарий является токсичным.

In [109]:
# Функция для предсказания токсичности нового комментария
def predict_toxicity(comment: str) -> float:
    # Предсказывает вероятность того, что комментарий является токсичным.
    # Входной комментарий -строка
    # Вероятность токсичности - число от 0 до 1 (1-высшая токсичность)

    # Преобразуем входной комментарий с использованием того же обученного векторизатора
    comment_vec = vectorizer.transform([comment])

    # Получаем вероятности для каждого класса (0 и 1)
    probabilities = model.predict_proba(comment_vec)

    # Возвращаем вероятность для класса 1 (токсичный комментарий)
    return probabilities[0][1]

# для примера:
print(predict_toxicity("How are you doing, I think it's going crazy shit?"))
print(predict_toxicity("Good day, are you shure it's ok?"))

0.959609421904435
0.1993496680447563


Предсказания совпадают с ожиданиями

# Задание 5

Поздравляю! Вы только что написали первый бейзлайн для определения токсичности комментариев! :)

А теперь посмотрим, насколько он этичен.

Попробуйте предсказать, токсичен ли комментарий «Apples are stupid». Потом предскажите, токсичен ли комментарий «I love apples».

In [110]:
comment1 = "Apples are stupid"
comment2 = "I love apples"

print(predict_toxicity(comment1))
print(predict_toxicity(comment2))

0.9941491240407159
0.2533215111840326


Предсказания снова совпали с ожиданием

# Задание 6

Если ваш алгоритм работает корректно, то комментарий «I love apples» должен быть определен как нетоксичный, а «Apples are stupid» — как токсичный.

А теперь перейдем к пониманию того, как модель принимает решения: модель присваивает каждому из примерно 58 000 слов коэффициент, причем более высокие коэффициенты обозначают слова, которые модель считает более токсичными. Выведите десять слов, которые считаются наиболее токсичными, а также их коэффициенты.

In [111]:
# Вывод десяти слов с наибольшими коэффициентами (считающихся наиболее токсичными)

# Получаем коэффициенты 
word_coefficients = model.coef_[0]

# Создадим обратный словарь: индекс -> слово
index_to_word = {index: word for word, index in vectorizer.vocabulary_.items()}

# Сопоставляем коэффициенты словам, используя обратный словарь
word_to_coeff= {
    index_to_word[i]: word_coefficients[i]
    for i in range(len(word_coefficients)) # Проходим по всем индексам коэффициентов
}

# Сортируем слова по убыванию коэффициентов из альтернативного словаря и берем топ-10
most_toxic_words = sorted(word_to_coeff.items(), key=lambda item: item[1], reverse=True)[:20]

for word, coeff in most_toxic_words:
    print(f"Слово: '{word}' | Коэффициент: {coeff:.4f}")


Слово: 'stupid' | Коэффициент: 5.9167
Слово: 'idiot' | Коэффициент: 4.9859
Слово: 'idiots' | Коэффициент: 4.4323
Слово: 'stupidity' | Коэффициент: 3.8801
Слово: 'dumb' | Коэффициент: 3.7934
Слово: 'pathetic' | Коэффициент: 3.6465
Слово: 'crap' | Коэффициент: 3.6392
Слово: 'ignorant' | Коэффициент: 3.5086
Слово: 'bitch' | Коэффициент: 3.4354
Слово: 'moron' | Коэффициент: 3.3648
Слово: 'damn' | Коэффициент: 3.3618
Слово: 'ridiculous' | Коэффициент: 3.2302
Слово: 'ass' | Коэффициент: 3.1135
Слово: 'garbage' | Коэффициент: 3.1027
Слово: 'idiotic' | Коэффициент: 3.0650
Слово: 'hypocrite' | Коэффициент: 2.9520
Слово: 'loser' | Коэффициент: 2.9430
Слово: 'fool' | Коэффициент: 2.8547
Слово: 'morons' | Коэффициент: 2.8435
Слово: 'silly' | Коэффициент: 2.7878


# Задание 7
Взгляните на самые токсичные слова из задания 6. Вызывают ли у вас удивление какие-нибудь из них? Есть ли слова, которых, кажется, не должно быть в списке?

Присутствуют слова токсичные,  но некоторые надо рассматривать в  контексте:
- pathetic - жалкий
- hypocrite - лицимер
- ignorant - невежественный

In [112]:
pathetic = "His pathetic attempt to win the argument only made the situation more embarrassing." #Бил лицимер, потомучто скрывает правду
hypocrite = "She accused him of being a hypocrite after he criticized her for something he himself had done." #Ему постоянно говорили, что он невежественный

print(predict_toxicity(pathetic))
print(predict_toxicity(hypocrite))


0.85908329173186
0.9570992347746421


# Задание 8

Давайте попробуем протестировать модель на ее предвзятость, например, к религии.

In [113]:
print(predict_toxicity("I have a christian friend"))
print(predict_toxicity("I have a muslim friend"))
print(predict_toxicity("I have a white friend"))
print(predict_toxicity("I have a black friend"))

0.4151673080399158
0.42155025913898503
0.3617148193271017
0.3433397813352252


Предсказания равномерны

# Задание 9 
Это может привести к нарушению справедливости модели, поскольку её предсказания будут зависеть не только от реальной токсичности комментария, но и от его тематики. Если определённые темы исторически или социально ассоциируются с негативными высказываниями в обучающей выборке (например, упоминание расы, религии или этнической принадлежности), модель может начать ошибочно считать такие слова маркерами токсичности, даже если контекст нейтральный. Это отражает предвзятость данных, на которых она обучалась, и может привести к дискриминационным предсказаниям.

# Задание 10
- Сбор более сбалансированного датасета

- Можно искусственно увеличить количество примеров нетоксичных комментариев на "чувствительные" темы 

- Фильтрация данных

# Задание 11

Этот фильтр перед тем как разбивать данные увеличит этичность

In [114]:
danger_themes = ['asian','atheist','black','buddhist','hindu','jewish','latino',
                   'muslim','other_religion','white','christian','bisexual','heterosexual',
                   'homosexual_gay_or_lesbian','intellectual_or_learning_disability','male',
                   'other_disability','other_gender', 'other_race_or_ethnicity', 'other_sexual_orientation',
                   'physical_disability','transgender', 'psychiatric_or_mental_illness']
mask = (data[danger_themes] == 0).all(axis=1)
data = data.loc[mask]

comments = data["comment_text"]
target = (data["target"]>0.7).astype(int)

comm_train, comm_test, target_train, target_test = train_test_split(
    comments, target, test_size=0.3, random_state=0
)


model = LogisticRegression(max_iter=2000)
model.fit(comments_train_vec, target_train)

print(predict_toxicity("I have a christian friend"))
print(predict_toxicity("I have a muslim friend"))
print(predict_toxicity("I have a white friend"))
print(predict_toxicity("I have a black friend"))

0.4151673080399158
0.42155025913898503
0.3617148193271017
0.3433397813352252


Предсказания стали качественнее